In [ ]:
%cd /content/drive/MyDrive/DUAN2/GPT2

/content/drive/MyDrive/DUAN2/GPT2


In [ ]:
import os
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm

In [ ]:
os.listdir("./train_data/Vietnamese Legal DS")

['bo-luat-dan-su.txt',
 'bo-luat-hang-hai.txt',
 'bo-luat-hinh-su.txt',
 'bo-luat-lao-dong.txt',
 'bo-luat-to-tung-dan-su.txt',
 'hien-phap.txt',
 'bo-luat-to-tung-hinh-su.txt',
 'luat-an-ninh-quoc-gia.txt',
 'luat-ban-hanh-van-ban-quy-pham-phap-luat.txt',
 'luat-bao-chi.txt',
 'luat-bao-hiem-xa-hoi.txt',
 'luat-bao-ve-cham-soc-va-giao-duc-tre-em.txt',
 'luat-bao-ve-suc-khoe-nhan-dan.txt',
 'luat-bao-ve-moi-truong.txt',
 'luat-bau-cu-dai-bieu-hdnd.txt',
 'luat-bao-ve-va-phat-trien-rung.txt',
 'luat-bau-cu-dai-bieu-quoc-hoi.txt',
 'luat-bien-gioi-quoc-gia.txt',
 'luat-cac-cong-cu-chuyen-nhuong.txt',
 'luat-binh-dang-gioi.txt',
 'luat-cai-cach-ruong-dat.txt',
 'luat-cac-to-chuc-tin-dung.txt',
 'luat-canh-tranh.txt',
 'luat-chung-khoan.txt',
 'luat-chuyen-giao-cong-nghe.txt',
 'luat-cong-an-nhan-dan.txt',
 'luat-cong-doan.txt',
 'luat-cong-chung.txt',
 'luat-cong-ty.txt',
 'luat-cong-nghe-thong-tin.txt',
 'luat-cu-tru.txt',
 'luat-dam-bao-quyen-tu-do-than-the-va-quyen-bat-kha-xam-pham-doi

In [ ]:
def get_data():
  data = []
  path = "./train_data/Vietnamese Legal DS"

  for file_name in os.listdir(path):
    with open(os.path.join(path,file_name),"r") as f:
      lines = f.readlines()
      for line in lines:
        line_text = line.strip()
        if line_text!="":
          data.append(line_text)
  return data

In [ ]:
print(len(get_data()))
get_data()[:5]

45472


['QUỐC HỘI',
 '******',
 'CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM',
 'Độc lập - Tự do - Hạnh phúc',
 '************']

In [ ]:
def train():
  tokenizer = GPT2Tokenizer.from_pretrained('NlpHUST/gpt2-vietnamese')
  model = TFGPT2LMHeadModel.from_pretrained('gpt2')

  text = get_data()[:10000]
  max_length = 256

  # Tokenize and create input/output pairs
  input_ids = []
  output_ids = []

  n = len(text)-1
  for i in tqdm(range(n)):
    # create pairs
    input_text = text[i]
    output_text = text[i+1]

    # encode
    input_tokenized = tokenizer.encode(input_text)
    output_tokenized = tokenizer.encode(output_text)

    # fixed length 256
    if len(input_tokenized) > max_length:
      input_tokenized = input_tokenized[:max_length]
    if len(output_tokenized) > max_length:
      output_tokenized = output_tokenized[:max_length]

    input_ids.append(input_tokenized)
    output_ids.append(output_tokenized)


  input_ids   = tf.keras.preprocessing.sequence.pad_sequences(tqdm(input_ids),maxlen=max_length, padding='post')
  output_ids  = tf.keras.preprocessing.sequence.pad_sequences(tqdm(output_ids), maxlen=max_length, padding='post')

  batch_size = 8
  epochs = 10
  print(epochs, batch_size)

  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

  model.compile(optimizer=optimizer, loss=model.hf_compute_loss)
  model.fit(input_ids, output_ids, batch_size=batch_size,epochs=epochs)
  # Save the trained model
  model.save_pretrained('chatbot_model')

In [ ]:
train()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
100%|██████████| 7999/7999 [00:00<00:00, 1662713.73it/s]


10 8
Epoch 1/10
1000/1000 [==============================] - 677s 644ms/step - loss: 0.8359
Epoch 2/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.7727
Epoch 3/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.7489
Epoch 4/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.7283
Epoch 5/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.7073
Epoch 6/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.6872
Epoch 7/10
1000/1000 [==============================] - 646s 646ms/step - loss: 0.6648
Epoch 8/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.6443
Epoch 9/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.6264
Epoch 10/10
1000/1000 [==============================] - 647s 647ms/step - loss: 0.6105


In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, pipeline
generator = pipeline('text-generation', model='chatbot_model', tokenizer='NlpHUST/gpt2-vietnamese')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at chatbot_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/854k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/512k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
while input != "":
  input_text = input("User: ")
  output_text = generator(input_text, max_length=200, pad_token_id=50256, num_return_sequences=1)
  print("Bot: " + output_text[0]['generated_text'])

User: thuế là
Bot: thuế là 18. Quy Nguyên thủ ký đổi đổi đổi đổi ký đổi đổi dụng đổi
User: thuế là gì
Bot: thuế là gì.----
User: 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyboardInterrupt: ignored